# Importando as bibliotecas

In [ ]:
import pandas as pd
import nltk
import numpy as np
import matplotlib.pyplot as plt

nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
nltk.download('averaged_perceptron_tagger_eng', quiet=True)
from db import connection_db as conndb
from db import filters
from tqdm.auto import tqdm

tqdm.pandas()

Conectando ao banco de dados do MongoDB

In [ ]:
uri = 'mongodb://localhost:27017/'
db_name = 'dadosVivamente'
col_name = 'dadosSemFiltros'

mongo_connection = conndb.MongoDBConnection(uri=uri, database_name=db_name, collection_name=col_name)
mongo_connection.connect()
collection = mongo_connection.collection

Aplicando pipeline para preparação dos dados

In [ ]:
collection_filters = filters.CollectionFilters(collection)

# Aplicando pipeline 1
collection_filters.apply_pipeline1('dadosComFiltrosIniciais')

# Aplicando pipeline 2 - pega os posts que tenha a data de publicação de no máximo 7 anos e no mínimo 2 anos
collection_filters.apply_pipeline2(7, 1, 'posts7anos1anos')

# Aplicando pipeline 3 - desenrola os posts em documentos individuais e cria novas colunas
collection_filters.apply_pipeline3('postsComBDIAndInfos')

# Aplicando pipeline 4 - aplica o filtro para selecionar apenas os posts que tem uma data de publicação válida
collection_filters.apply_pipeline4('postsComBDIAndInfosFiltroDataPosts')

collection = collection_filters.collection

In [ ]:
documentos = collection.find()
df = pd.DataFrame(list(documentos))

In [ ]:
df.head()

In [ ]:
# Lista de colunas que precisam ser convertidas
colunas_para_converter = [
    'pessimismo', 'tristeza', 'fracasso', 'prazer', 'culpa', 'punicao', 'estima',
    'critica', 'suicida', 'choro', 'agitacao', 'interesse', 'indecisao',
    'desvalorizacao', 'energia', 'sono', 'irritabilidade', 'apetite',
    'concentracao', 'fadiga', 'int_sexo', 'quantAmigos'
]

# Preencher os NaNs com 0
df[colunas_para_converter] = df[colunas_para_converter].fillna(0)

df[colunas_para_converter] = df[colunas_para_converter].astype('int64')

# Filtragem dos dados

In [ ]:
df.drop(columns=['_id', 'diaDaSemana', 'hora', 'minutos', 'diaDoMes', 'mes', 'ano'])

In [ ]:
df['data'] = df['postCreatedTime'].dt.date
df['data'] = pd.to_datetime(df['data'])

# EDA com filtro de 11 meses

In [ ]:
# Filtrar posts do ano de 2016
df_2017 = df[df['data'].dt.year == 2017]

In [ ]:
# Remover posts de dezembro de 2017
df_2017 = df_2017[~((df_2017['data'].dt.month == 12) & (df_2017['data'].dt.year == 2017))]

In [ ]:
# Contar quantos usuários postaram em 2017

niveis = [1, 2, 3]

for nivel in niveis:
    # Filtrar o DataFrame para o nível de suicida atual
    df_nivel = df_2017[df_2017['suicida'] == nivel]

    # Agrupar por sexo e contar quantos usuários únicos existem por sexo no nível atual
    usuarios_por_genero = df_nivel.groupby('sexo')['id_usuario'].nunique()

    # Plotar gráfico de barras para o nível atual
    ax = usuarios_por_genero.plot(kind='bar', figsize=(8, 6), color=['skyblue', 'lightcoral'])
    plt.title(f'Quantidade de Usuários por Sexo e Nível de Ideação Suicida {nivel}')
    plt.xlabel('Gênero')
    plt.ylabel('Número de Usuários')
    plt.xticks(rotation=0)

    # Adicionar os valores no topo de cada barra
    for p in ax.patches:
        ax.annotate(f'{int(p.get_height())}', (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha='center', va='center', xytext=(0, 5), textcoords='offset points')

    # Exibir o gráfico
    plt.show()

In [ ]:
df_2017 = df_2017[df_2017['suicida'] == 3]

In [ ]:
# Contar quantos posts cada usuário fez por dia
posts_por_dia = df_2017.groupby(['id_usuario', 'data']).size().reset_index(name='num_posts')

posts_por_dia.to_csv('posts_por_dia.csv', index=False)

# Exibir os dados
posts_por_dia.head()

In [ ]:
# Converter a coluna 'data' para datetime e criar uma coluna de 'mes'
posts_por_dia['data'] = pd.to_datetime(posts_por_dia['data'])
posts_por_dia['mes'] = posts_por_dia['data'].dt.to_period('M')

# Agrupar por 'id_usuario' e 'mes' para contar os posts por mês para cada usuário
posts_por_mes_usuario = posts_por_dia.groupby(['id_usuario', 'mes'])['num_posts'].sum().reset_index()

# Plotar gráficos de linha separados para cada usuário
for usuario in posts_por_mes_usuario['id_usuario'].unique():
    df_usuario = posts_por_mes_usuario[posts_por_mes_usuario['id_usuario'] == usuario]

    # Plotar gráfico de linha para o usuário atual
    # plt.figure(figsize=(10, 6))
    # plt.plot(df_usuario['mes'].astype(str), df_usuario['num_posts'], marker='o')
    # plt.title(f'Distribuição de posts por mês - Usuário {usuario}')
    # plt.xlabel('Mês')
    # plt.ylabel('Número de posts')
    # plt.xticks(rotation=90)
    # plt.grid(True)
    # 
    # # Exibir o gráfico
    # plt.show()

In [ ]:
posts_por_dia['semana'] = posts_por_dia['data'].dt.to_period('W')

# Agrupar por 'id_usuario' e 'semana' para contar os posts por semana para cada usuário
posts_por_semana_usuario = posts_por_dia.groupby(['id_usuario', 'semana'])['num_posts'].sum().reset_index()

for usuario in posts_por_semana_usuario['id_usuario'].unique():
    df_usuario = posts_por_semana_usuario[posts_por_semana_usuario['id_usuario'] == usuario]

    # Plotar gráfico de linha para o usuário atual
    # plt.figure(figsize=(10, 6))
    # plt.plot(df_usuario['semana'].astype(str), df_usuario['num_posts'], marker='o')
    # plt.title(f'Distribuição de posts por Semana - Usuário {usuario}')
    # plt.xlabel('Semana')
    # plt.ylabel('Número de posts')
    # plt.xticks(rotation=90)
    # plt.grid(True)
    # 
    # # Exibir o gráfico
    # plt.show()

# EDA com filtro de 6 meses

In [ ]:
# Passo 1: Filtrar dados após maio de 2017
df_2017 = df_2017[df_2017['data'].dt.month >= 5]

# Passo 2: Contar quantos posts cada usuário fez nesses meses
posts_por_usuario = df_2017.groupby('id_usuario').size().reset_index(name='num_posts')

# Passo 3: Filtrar usuários com 60 ou mais posts
usuarios_com_60_ou_mais_posts = posts_por_usuario[posts_por_usuario['num_posts'] >= 60]

# Exibir os usuários que têm 60 ou mais posts
print(usuarios_com_60_ou_mais_posts)

df_2017 = df_2017[df_2017['id_usuario'].isin(usuarios_com_60_ou_mais_posts['id_usuario'])]

# Exibir o DataFrame filtrado
df_2017.head()

In [ ]:
df_2017.to_csv('df_2017_filtro_6m.csv', index=False)

In [ ]:
# Contar quantos posts cada usuário fez nesses 6 meses
posts_por_dia = df_2017.groupby(['id_usuario', 'data']).size().reset_index(name='num_posts')

posts_por_dia.to_csv('posts_por_dia_filtro_6m.csv', index=False)

# Exibir os dados
posts_por_dia.head()

In [ ]:
# Converter a coluna 'data' para datetime e criar uma coluna de 'mes'
posts_por_dia['data'] = pd.to_datetime(posts_por_dia['data'])
posts_por_dia['mes'] = posts_por_dia['data'].dt.to_period('M')

# Agrupar por 'id_usuario' e 'mes' para contar os posts por mês para cada usuário
posts_por_mes_usuario = posts_por_dia.groupby(['id_usuario', 'mes'])['num_posts'].sum().reset_index()

# Plotar gráficos de linha separados para cada usuário
for usuario in posts_por_mes_usuario['id_usuario'].unique():
    df_usuario = posts_por_mes_usuario[posts_por_mes_usuario['id_usuario'] == usuario]

    # Plotar gráfico de linha para o usuário atual
    # plt.figure(figsize=(10, 6))
    # plt.plot(df_usuario['mes'].astype(str), df_usuario['num_posts'], marker='o')
    # plt.title(f'Distribuição de posts por mês - Usuário {usuario}')
    # plt.xlabel('Mês')
    # plt.ylabel('Número de posts')
    # plt.xticks(rotation=90)
    # plt.grid(True)
    # 
    # # Exibir o gráfico
    # plt.show()

In [ ]:
posts_por_dia['semana'] = posts_por_dia['data'].dt.to_period('W')

# Agrupar por 'id_usuario' e 'semana' para contar os posts por semana para cada usuário
posts_por_semana_usuario = posts_por_dia.groupby(['id_usuario', 'semana'])['num_posts'].sum().reset_index()

for usuario in posts_por_semana_usuario['id_usuario'].unique():
    df_usuario = posts_por_semana_usuario[posts_por_semana_usuario['id_usuario'] == usuario]

    # Plotar gráfico de linha para o usuário atual
    # plt.figure(figsize=(10, 6))
    # plt.plot(df_usuario['semana'].astype(str), df_usuario['num_posts'], marker='o')
    # plt.title(f'Distribuição de posts por mês - Usuário {usuario}')
    # plt.xlabel('Semana')
    # plt.ylabel('Número de posts')
    # plt.xticks(rotation=90)
    # plt.grid(True)
    # 
    # # Exibir o gráfico
    # plt.show()

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler

# Normalizar os dados para ter uma distribuição mais uniforme
scaler = StandardScaler()
posts_por_semana_usuario['num_posts_normalized'] = scaler.fit_transform(posts_por_semana_usuario[['num_posts']])

In [ ]:
posts_por_semana_usuario.head()

In [ ]:
# Usar Isolation Forest para detectar outliers na quantidade de posts semanais
isolation_forest = IsolationForest(contamination=0.01, random_state=42)
posts_por_semana_usuario['outlier'] = isolation_forest.fit_predict(posts_por_semana_usuario[['num_posts_normalized']])

# Filtrar os valores não outliers
posts_por_semana_sem_outliers = posts_por_semana_usuario[posts_por_semana_usuario['outlier'] == 1]

# Exibir os dados sem outliers
posts_por_semana_sem_outliers.head()

In [ ]:
posts_por_semana_sem_outliers.to_csv('posts_por_semana_sem_outliers.csv', index=False)

In [ ]:
# Agora você pode plotar os dados semanais sem outliers ou destacar os outliers
plt.figure(figsize=(10, 6))
plt.scatter(posts_por_semana_usuario['semana'].astype(str), posts_por_semana_usuario['num_posts_normalized'],
            c=posts_por_semana_usuario['outlier'], cmap='coolwarm')
plt.title('Detecção de Outliers na Quantidade de Posts por Semana')
plt.xlabel('Semana')
plt.ylabel('Número de Posts')
plt.xticks(rotation=90)
plt.grid(True)
plt.show()

# Decomposição sazonal individualmente

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

periodo = 30

# Passo 2: Iterar sobre cada usuário
for usuario in df_2017['id_usuario'].unique():

    # Filtrar o DataFrame para o usuário atual
    df_usuario = df_2017[df_2017['id_usuario'] == usuario]

    # Passo 3: Agrupar por data para contar o número de posts por dia
    # Preencher datas faltantes com 0
    posts_por_dia = df_usuario.groupby('data').size().reindex(pd.date_range(df_usuario['data'].min(),
                                                                            df_usuario['data'].max(), freq='D'),
                                                              fill_value=0)

    # Verificar se há observações suficientes para decomposição (>= 2 * periodo)
    if len(posts_por_dia) >= 2 * periodo:
        # Aplicar a decomposição sazonal
        try:
            resultado = seasonal_decompose(posts_por_dia, model='additive', period=periodo)

            # Passo 5: Plotar os componentes da decomposição
            fig = plt.figure(figsize=(8, 6))
            resultado.plot()
            plt.suptitle(f'Decomposição da Série Temporal - Usuário {usuario}', y=1.02)
            plt.show()

        except ValueError as e:
            print(f"Não foi possível decompor a série para o usuário {usuario}: {e}")
    else:
        print(
            f"Usuário {usuario} não tem observações suficientes para decomposição. Necessário: {2 * periodo}, disponível: {len(posts_por_dia)}")

In [ ]:
from sklearn.linear_model import LinearRegression

# Período de decomposição (ajuste conforme necessário)
periodo = 30

# Iterar sobre cada usuário
for usuario in df_2017['id_usuario'].unique():

    # Filtrar o DataFrame para o usuário atual
    df_usuario = df_2017[df_2017['id_usuario'] == usuario]

    # Agrupar por dia e contar o número de posts por dia
    posts_por_dia = df_usuario.groupby('data').size()

    # Verificar se há observações suficientes (>= 2 * periodo)
    if len(posts_por_dia) >= 2 * periodo:
        try:
            # Aplicar a decomposição sazonal
            resultado = seasonal_decompose(posts_por_dia, model='additive', period=periodo)

            # Extraindo a tendência da decomposição
            tendencia = resultado.trend.dropna()

            # Preparar dados para a regressão linear
            X = np.arange(len(tendencia)).reshape(-1, 1)
            y = tendencia.values  # Valores da tendência

            # Aplicar a Regressão Linear
            model = LinearRegression()
            model.fit(X, y)

            # Inclinação (slope) da tendência
            slope = model.coef_[0]

            # Exibir o resultado
            print(f'Inclinação (slope) da tendência para o usuário {usuario}: {slope}')

            # Plotar os dados e a linha de tendência ajustada
            plt.figure(figsize=(15, 12))
            plt.plot(tendencia.index, tendencia.values, label='Tendência Original')
            plt.plot(tendencia.index, model.predict(X), label='Tendência Ajustada (Regressão Linear)', linestyle='--')
            plt.title(f'Tendência para o Usuário {usuario}')
            plt.xlabel('Data')
            plt.ylabel('Tendência')
            plt.legend()
            plt.grid(True)
            plt.show()

        except ValueError as e:
            print(f"Erro ao decompor a série para o usuário {usuario}: {e}")
    else:
        print(
            f"Usuário {usuario} não tem observações suficientes para decomposição. Necessário: {2 * periodo}, disponível: {len(posts_por_dia)}")

In [ ]:
# Período de decomposição (ajuste conforme necessário)
periodo = 30

# Variáveis para contar tendências
usuarios_tendencia_para_cima = 0
usuarios_tendencia_para_baixo = 0

# Iterar sobre cada usuário
for usuario in df_2017['id_usuario'].unique():

    # Filtrar o DataFrame para o usuário atual
    df_usuario = df_2017[df_2017['id_usuario'] == usuario]

    # Agrupar por dia e contar o número de posts por dia
    posts_por_dia = df_usuario.groupby('data').size()

    # Verificar se há observações suficientes (>= 2 * periodo)
    if len(posts_por_dia) >= 2 * periodo:
        try:
            # Aplicar a decomposição sazonal
            resultado = seasonal_decompose(posts_por_dia, model='additive', period=periodo)

            # Extraindo a tendência da decomposição
            tendencia = resultado.trend.dropna()

            # Preparar dados para a regressão linear
            X = np.arange(len(tendencia)).reshape(-1, 1)
            y = tendencia.values  # Valores da tendência

            # Aplicar a Regressão Linear
            model = LinearRegression()
            model.fit(X, y)

            # Inclinação (slope) da tendência
            slope = model.coef_[0]

            # Verificar se a tendência é para cima ou para baixo
            if slope > 0:
                usuarios_tendencia_para_cima += 1
            else:
                usuarios_tendencia_para_baixo += 1

        except ValueError as e:
            print(f"Erro ao decompor a série para o usuário {usuario}: {e}")
    else:
        print(
            f"Usuário {usuario} não tem observações suficientes para decomposição. Necessário: {2 * periodo}, disponível: {len(posts_por_dia)}")

# Exibir os resultados
print(f"Usuários com tendência para cima: {usuarios_tendencia_para_cima}")
print(f"Usuários com tendência para baixo: {usuarios_tendencia_para_baixo}")

In [ ]:
# Criar gráfico de barras mostrando a contagem
labels = ['Tendência para cima', 'Tendência para baixo']
values = [usuarios_tendencia_para_cima, usuarios_tendencia_para_baixo]

plt.figure(figsize=(8, 6))
plt.bar(labels, values, color=['green', 'red'])
plt.title('Quantidade de Usuários com Tendência para Cima e para Baixo')
plt.ylabel('Número de Usuários')
plt.show()

In [ ]:
# Período de decomposição (ajuste conforme necessário)
periodo = 30

# Variáveis para contar e armazenar IDs
usuarios_tendencia_para_cima = []
usuarios_tendencia_para_baixo = []

# Iterar sobre cada usuário
for usuario in df_2017['id_usuario'].unique():

    # Filtrar o DataFrame para o usuário atual
    df_usuario = df_2017[df_2017['id_usuario'] == usuario]

    # Agrupar por dia e contar o número de posts por dia
    posts_por_dia = df_usuario.groupby('data').size()

    # Verificar se há observações suficientes (>= 2 * periodo)
    if len(posts_por_dia) >= 2 * periodo:
        try:
            # Aplicar a decomposição sazonal
            resultado = seasonal_decompose(posts_por_dia, model='additive', period=periodo)

            # Extraindo a tendência da decomposição
            tendencia = resultado.trend.dropna()  # Remover valores NaN da tendência

            # Preparar dados para a regressão linear
            X = np.arange(len(tendencia)).reshape(-1, 1)  # Dias consecutivos
            y = tendencia.values  # Valores da tendência

            # Aplicar a Regressão Linear
            model = LinearRegression()
            model.fit(X, y)

            # Inclinação (slope) da tendência
            slope = model.coef_[0]

            # Verificar se a tendência é para cima ou para baixo e armazenar o ID
            if slope > 0:
                usuarios_tendencia_para_cima.append(usuario)
            else:
                usuarios_tendencia_para_baixo.append(usuario)

        except ValueError as e:
            print(f"Erro ao decompor a série para o usuário {usuario}: {e}")
    else:
        print(
            f"Usuário {usuario} não tem observações suficientes para decomposição. Necessário: {2 * periodo}, disponível: {len(posts_por_dia)}")

# Exibir os resultados
print(f"Usuários com tendência para cima: {len(usuarios_tendencia_para_cima)}")
print(f"Usuários com tendência para baixo: {len(usuarios_tendencia_para_baixo)}")

# Exibir os IDs
print("IDs dos usuários com tendência para cima:", usuarios_tendencia_para_cima)
print("IDs dos usuários com tendência para baixo:", usuarios_tendencia_para_baixo)

# Criar gráfico de barras mostrando a contagem
labels = ['Tendência para cima', 'Tendência para baixo']
values = [len(usuarios_tendencia_para_cima), len(usuarios_tendencia_para_baixo)]

plt.figure(figsize=(8, 6))
plt.bar(labels, values, color=['green', 'red'])
plt.title('Quantidade de Usuários com Tendência para Cima e para Baixo')
plt.ylabel('Número de Usuários')
plt.show()

In [ ]:
# Lista de colunas relacionadas ao BDI-II
colunas_bdi_ii = [
    'pessimismo', 'tristeza', 'fracasso', 'prazer', 'culpa', 'punicao',
    'estima', 'critica', 'suicida', 'choro', 'agitacao', 'interesse',
    'indecisao', 'desvalorizacao', 'energia', 'sono', 'irritabilidade',
    'apetite', 'concentracao', 'fadiga', 'int_sexo'
]

# Filtrar os usuários com tendência para baixo
df_tendencia_baixo = df_2017[df_2017['id_usuario'].isin(usuarios_tendencia_para_baixo)]

# Selecionar apenas as colunas do BDI-II para esses usuários
df_bdi_ii = df_tendencia_baixo[colunas_bdi_ii]

# Verificar as respostas mais frequentes em cada coluna do BDI-II
respostas_comuns = df_bdi_ii.mode().iloc[0]  # Obtém a moda (resposta mais comum) para cada coluna

# Exibir as respostas mais comuns para os usuários com tendência para baixo
print("Respostas mais comuns do BDI-II para os usuários com tendência para baixo:")
print(respostas_comuns)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns

# Lista de colunas relacionadas ao BDI-II
colunas_bdi_ii = [
    'pessimismo', 'tristeza', 'fracasso', 'prazer', 'culpa', 'punicao',
    'estima', 'critica', 'suicida', 'choro', 'agitacao', 'interesse',
    'indecisao', 'desvalorizacao', 'energia', 'sono', 'irritabilidade',
    'apetite', 'concentracao', 'fadiga', 'int_sexo'
]

# Criar a variável alvo (target), onde 1 indica tendência para baixo e 0 para cima
df_2017['tendencia'] = df_2017['id_usuario'].apply(lambda x: 1 if x in usuarios_tendencia_para_baixo else 0)

# Filtrar os dados, mantendo apenas as colunas do BDI-II e a variável alvo (tendencia)
df_bdi_ii_model = df_2017[colunas_bdi_ii + ['tendencia']]

# Remover possíveis NaN (se houver)
df_bdi_ii_model = df_bdi_ii_model.dropna()

# Separar os dados em features (X) e target (y)
X = df_bdi_ii_model[colunas_bdi_ii]
y = df_bdi_ii_model['tendencia']

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Treinar o modelo Random Forest
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = clf.predict(X_test)

# Avaliar o modelo
print("Acurácia do Modelo:", accuracy_score(y_test, y_pred))
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred))

# Matriz de Confusão
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 4))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Matriz de Confusão')
plt.ylabel('Valor Verdadeiro')
plt.xlabel('Previsão')
plt.show()

# Importância das variáveis
importances = clf.feature_importances_
indices = np.argsort(importances)[::-1]

# Plotar a importância das features
plt.figure(figsize=(10, 6))
plt.title('Importância das Features (BDI-II)')
plt.bar(range(X.shape[1]), importances[indices], align='center')
plt.xticks(range(X.shape[1]), [colunas_bdi_ii[i] for i in indices], rotation=90)
plt.ylabel('Importância')
plt.show()

In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

# Lista de colunas relacionadas ao BDI-II
colunas_bdi_ii = [
    'pessimismo', 'tristeza', 'fracasso', 'prazer', 'culpa', 'punicao',
    'estima', 'critica', 'suicida', 'choro', 'agitacao', 'interesse',
    'indecisao', 'desvalorizacao', 'energia', 'sono', 'irritabilidade',
    'apetite', 'concentracao', 'fadiga', 'int_sexo'
]

# Criar a variável alvo (target), onde 1 indica tendência para baixo e 0 para cima
df_2017['tendencia'] = df_2017['id_usuario'].apply(lambda x: 1 if x in usuarios_tendencia_para_baixo else 0)

# Agregar as respostas do BDI-II por usuário, calculando a média das respostas para cada usuário
df_aggregated = df_2017.groupby('id_usuario')[colunas_bdi_ii].mean().reset_index()

# Adicionar a coluna 'tendencia' agregada por usuário
df_aggregated = df_aggregated.merge(df_2017[['id_usuario', 'tendencia']].drop_duplicates(), on='id_usuario')

# Remover possíveis NaN (se houver)
df_aggregated = df_aggregated.dropna()

# Separar os dados em features (X) e target (y)
X = df_aggregated[colunas_bdi_ii]
y = df_aggregated['tendencia']

# Dividir os dados em treino e teste (estratificado por usuário)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Treinar o modelo Random Forest
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = clf.predict(X_test)

# Avaliar o modelo
print("Acurácia do Modelo:", accuracy_score(y_test, y_pred))
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred))

# Matriz de Confusão
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 4))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Matriz de Confusão')
plt.ylabel('Valor Verdadeiro')
plt.xlabel('Previsão')
plt.show()

# Importância das variáveis
importances = clf.feature_importances_
indices = np.argsort(importances)[::-1]

# Plotar a importância das features
plt.figure(figsize=(10, 6))
plt.title('Importância das Features (BDI-II)')
plt.bar(range(X.shape[1]), importances[indices], align='center')
plt.xticks(range(X.shape[1]), [colunas_bdi_ii[i] for i in indices], rotation=90)
plt.ylabel('Importância')
plt.show()